# Part 6: Algorithm Comparison and Conclusion

In this final notebook, we'll compare all the RL algorithms we've learned and provide a comprehensive summary of the tutorial.

## What You'll Learn
- Side-by-side comparison of all algorithms
- Performance benchmarks
- When to use which algorithm
- Complete summary of RL concepts
- Next steps for further learning

Let's begin!

## Setup

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import defaultdict

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
np.random.seed(42)

print("Setup complete!")

In [ ]:
# Create environment
env = gym.make("FrozenLake-v1", is_slippery=True)

n_states = env.observation_space.n
n_actions = env.action_space.n
action_names = ['LEFT', 'DOWN', 'RIGHT', 'UP']
action_arrows = ['←', '↓', '→', '↑']

print(f"FrozenLake: {n_states} states, {n_actions} actions")

In [ ]:
# Extract MDP for DP methods
def extract_mdp(env):
    n_s = env.observation_space.n
    n_a = env.action_space.n
    P = np.zeros((n_s, n_a, n_s))
    R = np.zeros((n_s, n_a))
    for s in range(n_s):
        for a in range(n_a):
            for prob, next_s, reward, done in env.unwrapped.P[s][a]:
                P[s, a, next_s] += prob
                R[s, a] += prob * reward
    return P, R

P, R = extract_mdp(env)

In [ ]:
# Visualization functions
def plot_policy(Q, title="Policy", ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
    
    desc = env.unwrapped.desc.astype(str)
    nrow, ncol = desc.shape
    colors = {'S': 'lightblue', 'F': 'white', 'H': 'lightcoral', 'G': 'lightgreen'}
    
    for i in range(nrow):
        for j in range(ncol):
            state = i * ncol + j
            cell = desc[i, j]
            
            rect = plt.Rectangle((j, nrow-1-i), 1, 1, fill=True,
                                 facecolor=colors.get(cell, 'white'), edgecolor='black')
            ax.add_patch(rect)
            
            best_action = np.argmax(Q[state])
            
            if cell not in ['H', 'G']:
                ax.text(j + 0.5, nrow - 1 - i + 0.5, 
                       f'{cell}\n{action_arrows[best_action]}',
                       ha='center', va='center', fontsize=12, fontweight='bold')
            else:
                ax.text(j + 0.5, nrow - 1 - i + 0.5, cell,
                       ha='center', va='center', fontsize=12, fontweight='bold')
    
    ax.set_xlim(0, ncol)
    ax.set_ylim(0, nrow)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=11)
    return ax

---
# 1. Implement All Algorithms

Let's implement all the algorithms we've learned in one place.

In [ ]:
# =====================
# DYNAMIC PROGRAMMING
# =====================

def policy_iteration(P, R, gamma, theta=1e-8):
    """Policy Iteration (DP method, requires model)."""
    n_states, n_actions = R.shape
    
    # Initialize random policy
    policy = np.ones((n_states, n_actions)) / n_actions
    
    iterations = 0
    while True:
        # Policy Evaluation
        V = np.zeros(n_states)
        while True:
            V_new = np.zeros(n_states)
            for s in range(n_states):
                for a in range(n_actions):
                    V_new[s] += policy[s, a] * (R[s, a] + gamma * np.sum(P[s, a] * V))
            if np.max(np.abs(V_new - V)) < theta:
                break
            V = V_new
        
        # Policy Improvement
        new_policy = np.zeros((n_states, n_actions))
        for s in range(n_states):
            q_values = [R[s, a] + gamma * np.sum(P[s, a] * V) for a in range(n_actions)]
            new_policy[s, np.argmax(q_values)] = 1.0
        
        iterations += 1
        if np.array_equal(new_policy, policy):
            break
        policy = new_policy
    
    # Compute Q from V
    Q = np.zeros((n_states, n_actions))
    for s in range(n_states):
        for a in range(n_actions):
            Q[s, a] = R[s, a] + gamma * np.sum(P[s, a] * V)
    
    return Q, iterations


def value_iteration(P, R, gamma, theta=1e-8):
    """Value Iteration (DP method, requires model)."""
    n_states, n_actions = R.shape
    V = np.zeros(n_states)
    
    iterations = 0
    while True:
        V_new = np.zeros(n_states)
        for s in range(n_states):
            V_new[s] = np.max([R[s, a] + gamma * np.sum(P[s, a] * V) for a in range(n_actions)])
        
        iterations += 1
        if np.max(np.abs(V_new - V)) < theta:
            break
        V = V_new
    
    # Extract Q
    Q = np.zeros((n_states, n_actions))
    for s in range(n_states):
        for a in range(n_actions):
            Q[s, a] = R[s, a] + gamma * np.sum(P[s, a] * V)
    
    return Q, iterations

In [ ]:
# =====================
# MONTE CARLO
# =====================

def mc_control(env, gamma, n_episodes, epsilon=0.1, epsilon_decay=0.99999, min_epsilon=0.01):
    """Monte Carlo Control with ε-greedy (model-free)."""
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    Q = np.zeros((n_states, n_actions))
    returns_sum = np.zeros((n_states, n_actions))
    returns_count = np.zeros((n_states, n_actions))
    episode_rewards = []
    
    for episode_num in range(n_episodes):
        # Generate episode
        episode = []
        state, _ = env.reset()
        done = False
        
        while not done:
            if np.random.random() < epsilon:
                action = np.random.randint(n_actions)
            else:
                action = np.argmax(Q[state])
            
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode.append((state, action, reward))
            state = next_state
            done = terminated or truncated
        
        episode_rewards.append(sum(r for _, _, r in episode))
        
        # Update Q
        sa_visited = set()
        G = 0
        for t in reversed(range(len(episode))):
            s, a, r = episode[t]
            G = gamma * G + r
            if (s, a) not in sa_visited:
                sa_visited.add((s, a))
                returns_sum[s, a] += G
                returns_count[s, a] += 1
                Q[s, a] = returns_sum[s, a] / returns_count[s, a]
        
        epsilon = max(min_epsilon, epsilon * epsilon_decay)
    
    return Q, episode_rewards

In [ ]:
# =====================
# TEMPORAL DIFFERENCE
# =====================

def sarsa(env, gamma, alpha, n_episodes, epsilon=0.1, epsilon_decay=0.99999, min_epsilon=0.01):
    """SARSA: On-policy TD Control (model-free)."""
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    Q = np.zeros((n_states, n_actions))
    episode_rewards = []
    
    def eps_greedy(state, eps):
        if np.random.random() < eps:
            return np.random.randint(n_actions)
        return np.argmax(Q[state])
    
    for _ in range(n_episodes):
        state, _ = env.reset()
        action = eps_greedy(state, epsilon)
        total_reward = 0
        done = False
        
        while not done:
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            next_action = eps_greedy(next_state, epsilon)
            
            # SARSA update
            td_target = reward + gamma * Q[next_state, next_action] * (not done)
            Q[state, action] += alpha * (td_target - Q[state, action])
            
            state, action = next_state, next_action
            total_reward += reward
        
        episode_rewards.append(total_reward)
        epsilon = max(min_epsilon, epsilon * epsilon_decay)
    
    return Q, episode_rewards


def q_learning(env, gamma, alpha, n_episodes, epsilon=0.1, epsilon_decay=0.99999, min_epsilon=0.01):
    """Q-Learning: Off-policy TD Control (model-free)."""
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    Q = np.zeros((n_states, n_actions))
    episode_rewards = []
    
    for _ in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        done = False
        
        while not done:
            # ε-greedy action selection
            if np.random.random() < epsilon:
                action = np.random.randint(n_actions)
            else:
                action = np.argmax(Q[state])
            
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Q-Learning update
            td_target = reward + gamma * np.max(Q[next_state]) * (not done)
            Q[state, action] += alpha * (td_target - Q[state, action])
            
            state = next_state
            total_reward += reward
        
        episode_rewards.append(total_reward)
        epsilon = max(min_epsilon, epsilon * epsilon_decay)
    
    return Q, episode_rewards

---
# 2. Run All Algorithms

In [ ]:
# Parameters
gamma = 0.99
n_episodes_mf = 100000  # For model-free methods

results = {}

print("Running All Algorithms")
print("=" * 60)

# Policy Iteration
print("\n1. Policy Iteration (DP)...")
start = time.time()
Q_pi, iters_pi = policy_iteration(P, R, gamma)
time_pi = time.time() - start
results['Policy Iteration'] = {'Q': Q_pi, 'time': time_pi, 'iterations': iters_pi}
print(f"   Done in {time_pi:.4f}s, {iters_pi} iterations")

# Value Iteration
print("\n2. Value Iteration (DP)...")
start = time.time()
Q_vi, iters_vi = value_iteration(P, R, gamma)
time_vi = time.time() - start
results['Value Iteration'] = {'Q': Q_vi, 'time': time_vi, 'iterations': iters_vi}
print(f"   Done in {time_vi:.4f}s, {iters_vi} iterations")

# Monte Carlo
print(f"\n3. Monte Carlo ({n_episodes_mf} episodes)...")
start = time.time()
Q_mc, rewards_mc = mc_control(env, gamma, n_episodes_mf, epsilon=1.0, epsilon_decay=0.99995)
time_mc = time.time() - start
results['Monte Carlo'] = {'Q': Q_mc, 'time': time_mc, 'rewards': rewards_mc}
print(f"   Done in {time_mc:.2f}s")

# SARSA
print(f"\n4. SARSA ({n_episodes_mf} episodes)...")
start = time.time()
Q_sarsa, rewards_sarsa = sarsa(env, gamma, alpha=0.1, n_episodes=n_episodes_mf, 
                                epsilon=1.0, epsilon_decay=0.99995)
time_sarsa = time.time() - start
results['SARSA'] = {'Q': Q_sarsa, 'time': time_sarsa, 'rewards': rewards_sarsa}
print(f"   Done in {time_sarsa:.2f}s")

# Q-Learning
print(f"\n5. Q-Learning ({n_episodes_mf} episodes)...")
start = time.time()
Q_qlearn, rewards_qlearn = q_learning(env, gamma, alpha=0.1, n_episodes=n_episodes_mf,
                                       epsilon=1.0, epsilon_decay=0.99995)
time_qlearn = time.time() - start
results['Q-Learning'] = {'Q': Q_qlearn, 'time': time_qlearn, 'rewards': rewards_qlearn}
print(f"   Done in {time_qlearn:.2f}s")

print("\n" + "=" * 60)
print("All algorithms complete!")

---
# 3. Evaluate All Policies

In [ ]:
def evaluate_policy(env, Q, n_episodes=10000):
    """Evaluate a greedy policy derived from Q."""
    rewards = []
    for _ in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        done = False
        while not done:
            action = np.argmax(Q[state])
            state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            done = terminated or truncated
        rewards.append(total_reward)
    return np.array(rewards)

print("Evaluating All Policies (10,000 episodes each)")
print("=" * 60)

for name in results:
    rewards = evaluate_policy(env, results[name]['Q'])
    results[name]['success_rate'] = np.mean(rewards) * 100
    results[name]['eval_rewards'] = rewards
    print(f"{name}: Success rate = {results[name]['success_rate']:.2f}%")

---
# 4. Comprehensive Comparison

In [ ]:
# Compare Q-values with optimal (Policy Iteration is our reference)
Q_optimal = results['Policy Iteration']['Q']

print("Q-value Accuracy (Mean Absolute Error vs Optimal)")
print("=" * 60)

for name in results:
    mae = np.mean(np.abs(results[name]['Q'] - Q_optimal))
    results[name]['q_mae'] = mae
    print(f"{name}: MAE = {mae:.6f}")

In [ ]:
# Create comprehensive comparison visualization
fig = plt.figure(figsize=(18, 12))

# 1. Success rates bar chart
ax1 = fig.add_subplot(2, 3, 1)
names = list(results.keys())
success_rates = [results[n]['success_rate'] for n in names]
colors = ['#2ecc71', '#27ae60', '#3498db', '#e74c3c', '#9b59b6']
bars = ax1.bar(range(len(names)), success_rates, color=colors, edgecolor='black')
ax1.set_xticks(range(len(names)))
ax1.set_xticklabels([n.replace(' ', '\n') for n in names], fontsize=9)
ax1.set_ylabel('Success Rate (%)')
ax1.set_title('Policy Performance')
ax1.set_ylim(0, 100)
for bar, rate in zip(bars, success_rates):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{rate:.1f}%', ha='center', fontsize=9, fontweight='bold')

# 2. Training time bar chart
ax2 = fig.add_subplot(2, 3, 2)
times = [results[n]['time'] for n in names]
bars = ax2.bar(range(len(names)), times, color=colors, edgecolor='black')
ax2.set_xticks(range(len(names)))
ax2.set_xticklabels([n.replace(' ', '\n') for n in names], fontsize=9)
ax2.set_ylabel('Time (seconds)')
ax2.set_title('Training Time')
for bar, t in zip(bars, times):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(times)*0.02,
            f'{t:.2f}s', ha='center', fontsize=9, fontweight='bold')

# 3. Q-value accuracy
ax3 = fig.add_subplot(2, 3, 3)
maes = [results[n]['q_mae'] for n in names]
bars = ax3.bar(range(len(names)), maes, color=colors, edgecolor='black')
ax3.set_xticks(range(len(names)))
ax3.set_xticklabels([n.replace(' ', '\n') for n in names], fontsize=9)
ax3.set_ylabel('Mean Absolute Error')
ax3.set_title('Q-value Accuracy (vs Optimal)')

# 4-8. Policies side by side
for idx, name in enumerate(names):
    ax = fig.add_subplot(2, 5, 6 + idx)
    plot_policy(results[name]['Q'], title=name, ax=ax)

plt.suptitle('Comprehensive Algorithm Comparison', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Learning curves for model-free methods
fig, ax = plt.subplots(figsize=(12, 5))

window = 1000
model_free = ['Monte Carlo', 'SARSA', 'Q-Learning']
colors_mf = ['#3498db', '#e74c3c', '#9b59b6']

for name, color in zip(model_free, colors_mf):
    rewards = results[name]['rewards']
    smooth = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax.plot(smooth, label=name, color=color, alpha=0.8)

ax.set_xlabel('Episode')
ax.set_ylabel(f'Reward (moving avg, window={window})')
ax.set_title('Learning Curves: Model-Free Methods')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# 5. Summary Table

In [ ]:
# Create summary table
print("\n" + "=" * 90)
print("ALGORITHM COMPARISON SUMMARY")
print("=" * 90)
print(f"{'Algorithm':<20} {'Type':<15} {'Model':<12} {'Success':<10} {'Time':<12} {'Q-MAE':<10}")
print("-" * 90)

algo_info = {
    'Policy Iteration': ('DP', 'Required'),
    'Value Iteration': ('DP', 'Required'),
    'Monte Carlo': ('MC', 'Free'),
    'SARSA': ('TD (On)', 'Free'),
    'Q-Learning': ('TD (Off)', 'Free')
}

for name in results:
    algo_type, model = algo_info[name]
    success = results[name]['success_rate']
    time_taken = results[name]['time']
    mae = results[name]['q_mae']
    print(f"{name:<20} {algo_type:<15} {model:<12} {success:>6.2f}%   {time_taken:>8.4f}s   {mae:>8.6f}")

print("=" * 90)

---
# 6. When to Use Which Algorithm?

## Decision Guide

```
Do you have a complete model of the environment?
│
├── YES → Use Dynamic Programming
│         ├── Policy Iteration: Fewer iterations, more work per iteration
│         └── Value Iteration: More iterations, less work per iteration
│
└── NO → Use Model-Free Methods
         │
         ├── Do episodes terminate?
         │   ├── YES → Can use Monte Carlo or TD
         │   └── NO → Must use TD methods
         │
         └── Do you want to learn the optimal policy?
             ├── YES → Q-Learning (off-policy)
             └── Policy you're following → SARSA (on-policy)
```

In [ ]:
# Create decision flowchart visualization
fig, ax = plt.subplots(figsize=(14, 8))

# Draw boxes
boxes = [
    {'pos': (0.5, 0.9), 'text': 'Start: Choose RL Algorithm', 'color': 'lightgray'},
    {'pos': (0.5, 0.75), 'text': 'Have complete\nmodel (P, R)?', 'color': 'lightyellow'},
    {'pos': (0.2, 0.55), 'text': 'Dynamic\nProgramming', 'color': 'lightgreen'},
    {'pos': (0.8, 0.55), 'text': 'Model-Free\nMethods', 'color': 'lightblue'},
    {'pos': (0.1, 0.35), 'text': 'Policy\nIteration', 'color': '#2ecc71'},
    {'pos': (0.3, 0.35), 'text': 'Value\nIteration', 'color': '#27ae60'},
    {'pos': (0.65, 0.35), 'text': 'Episodes\nterminate?', 'color': 'lightyellow'},
    {'pos': (0.5, 0.15), 'text': 'Monte Carlo', 'color': '#3498db'},
    {'pos': (0.95, 0.35), 'text': 'TD Methods', 'color': 'lightblue'},
    {'pos': (0.8, 0.15), 'text': 'SARSA\n(on-policy)', 'color': '#e74c3c'},
    {'pos': (1.0, 0.15), 'text': 'Q-Learning\n(off-policy)', 'color': '#9b59b6'},
]

for box in boxes:
    rect = plt.Rectangle((box['pos'][0]-0.08, box['pos'][1]-0.06), 0.16, 0.12,
                         facecolor=box['color'], edgecolor='black', linewidth=2)
    ax.add_patch(rect)
    ax.text(box['pos'][0], box['pos'][1], box['text'], ha='center', va='center',
           fontsize=9, fontweight='bold')

# Draw arrows with labels
arrows = [
    ((0.5, 0.84), (0.5, 0.81), ''),
    ((0.42, 0.69), (0.28, 0.61), 'Yes'),
    ((0.58, 0.69), (0.72, 0.61), 'No'),
    ((0.15, 0.49), (0.12, 0.41), ''),
    ((0.25, 0.49), (0.28, 0.41), ''),
    ((0.73, 0.49), (0.58, 0.41), ''),
    ((0.87, 0.49), (0.93, 0.41), 'No'),
    ((0.57, 0.29), (0.52, 0.21), 'Yes'),
    ((0.93, 0.29), (0.85, 0.21), ''),
    ((0.97, 0.29), (0.98, 0.21), ''),
]

for start, end, label in arrows:
    ax.annotate('', xy=end, xytext=start,
               arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    if label:
        mid = ((start[0]+end[0])/2, (start[1]+end[1])/2)
        ax.text(mid[0]+0.02, mid[1]+0.02, label, fontsize=9, fontweight='bold', color='darkblue')

ax.set_xlim(-0.05, 1.15)
ax.set_ylim(0.05, 1.0)
ax.axis('off')
ax.set_title('Algorithm Selection Guide', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

---
# 7. Complete RL Concepts Summary

## Core Concepts

| Concept | Definition | Formula |
|---------|------------|----------|
| **State** | Current situation | $s \in S$ |
| **Action** | Decision to take | $a \in A$ |
| **Reward** | Immediate feedback | $R_t$ |
| **Return** | Cumulative discounted reward | $G_t = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$ |
| **Policy** | Behavior strategy | $\pi(a|s) = P[A_t=a|S_t=s]$ |
| **State Value** | Expected return from state | $V^\pi(s) = E_\pi[G_t|S_t=s]$ |
| **Action Value** | Expected return from (state, action) | $Q^\pi(s,a) = E_\pi[G_t|S_t=s, A_t=a]$ |

## Bellman Equations

| Equation | Purpose | Form |
|----------|---------|------|
| **Bellman Expectation (V)** | Value of policy | $V^\pi(s) = \sum_a \pi(a|s)[R_s^a + \gamma \sum_{s'} P_{ss'}^a V^\pi(s')]$ |
| **Bellman Expectation (Q)** | Q of policy | $Q^\pi(s,a) = R_s^a + \gamma \sum_{s'} P_{ss'}^a V^\pi(s')$ |
| **Bellman Optimality (V)** | Optimal value | $V^*(s) = \max_a[R_s^a + \gamma \sum_{s'} P_{ss'}^a V^*(s')]$ |
| **Bellman Optimality (Q)** | Optimal Q | $Q^*(s,a) = R_s^a + \gamma \sum_{s'} P_{ss'}^a \max_{a'} Q^*(s',a')$ |

## Algorithm Summary

### Dynamic Programming (Model-Based)

| Algorithm | Key Idea | Update |
|-----------|----------|--------|
| **Policy Iteration** | Alternate eval & improve | Full policy evaluation |
| **Value Iteration** | One-step lookahead | $V(s) = \max_a[R + \gamma \sum P \cdot V]$ |

### Monte Carlo (Model-Free)

| Aspect | Description |
|--------|-------------|
| **Learns from** | Complete episodes |
| **Update** | After episode ends |
| **Uses** | Actual returns |
| **Variance** | High |
| **Bias** | None |

### Temporal Difference (Model-Free)

| Algorithm | Type | Update |
|-----------|------|--------|
| **TD(0)** | Prediction | $V(s) \leftarrow V(s) + \alpha[r + \gamma V(s') - V(s)]$ |
| **SARSA** | On-policy | $Q(s,a) \leftarrow Q(s,a) + \alpha[r + \gamma Q(s',a') - Q(s,a)]$ |
| **Q-Learning** | Off-policy | $Q(s,a) \leftarrow Q(s,a) + \alpha[r + \gamma \max Q(s',\cdot) - Q(s,a)]$ |

---
# 8. Limitations and Next Steps

## Limitations of Tabular Methods

All methods in this tutorial are **tabular**: they maintain a table of values for each state (or state-action pair).

**Problems with large state spaces:**
- Memory: Can't store a table with millions of entries
- Generalization: Each state learned independently
- Continuous states: Infinite states, can't enumerate

## What's Next: Function Approximation

Instead of tables, use **function approximators** (like neural networks):

$$V(s) \approx V(s; \theta)$$
$$Q(s, a) \approx Q(s, a; \theta)$$

Where $\theta$ are learnable parameters.

## Deep Reinforcement Learning

Combining RL with deep neural networks:

- **DQN** (Deep Q-Network): Q-Learning + Neural Network
- **Policy Gradient**: Directly optimize policy parameters
- **Actor-Critic**: Combine policy and value learning
- **PPO, SAC, TD3**: State-of-the-art algorithms

## Resources for Further Learning

1. **Sutton & Barto** - "Reinforcement Learning: An Introduction" (free online)
2. **David Silver's RL Course** - YouTube lectures from DeepMind
3. **OpenAI Spinning Up** - Practical Deep RL tutorial
4. **Stable Baselines3** - Ready-to-use RL algorithms in Python

---
# Congratulations!

You have completed this comprehensive Reinforcement Learning tutorial!

## What You've Learned

1. **Fundamentals** (Notebook 01)
   - What makes RL unique
   - Agent-environment interaction
   - States, actions, rewards, policies

2. **Mathematical Framework** (Notebook 02)
   - Markov Decision Processes
   - Bellman Equations
   - Optimal value functions

3. **Dynamic Programming** (Notebook 03)
   - Policy Evaluation
   - Policy Iteration
   - Value Iteration

4. **Monte Carlo Methods** (Notebook 04)
   - Learning from episodes
   - First-visit vs Every-visit
   - MC Control

5. **Temporal Difference** (Notebook 05)
   - TD(0) Prediction
   - SARSA (On-policy)
   - Q-Learning (Off-policy)

6. **Comparison & Summary** (This Notebook)
   - All algorithms compared
   - When to use what
   - Next steps

In [ ]:
print("="*70)
print("   CONGRATULATIONS! You've completed the RL Tutorial!")
print("="*70)
print("\n📚 Notebooks completed:")
print("   01. Introduction to Reinforcement Learning")
print("   02. MDPs and Bellman Equations")
print("   03. Dynamic Programming")
print("   04. Monte Carlo Methods")
print("   05. Temporal Difference Learning")
print("   06. Algorithm Comparison (this one!)")
print("\n🎯 Algorithms mastered:")
print("   • Policy Iteration")
print("   • Value Iteration")
print("   • Monte Carlo Control")
print("   • SARSA")
print("   • Q-Learning")
print("\n🚀 Next steps:")
print("   • Try different environments (CartPole, MountainCar, etc.)")
print("   • Learn about Deep RL (DQN, Policy Gradients)")
print("   • Read Sutton & Barto's book for deeper understanding")
print("   • Implement your own RL agent for a real problem!")
print("\n" + "="*70)
print("   Happy Learning! 🎉")
print("="*70)